# tests tab-NTV

In [1]:
import sys
sys.path

['C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Tests',
 'C:\\Users\\phili\\github\\NTV\\json_ntv',
 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\observation',
 'C:\\Users\\phili\\github\\NTV',
 'C:\\Users\\phili\\github\\Environmental-Sensing\\python',
 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Tests',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\python310.zip',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\DLLs',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309',
 '',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages\\win32',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\phili\\anaconda3\\envs\\obser

In [132]:
import folium

class carto():
    def __init__(self, location=None, zoom_start=6):
        self.map = folium.Map(location=location, zoom_start=zoom_start)

   
    def add_marker_list(self, location, popup=None, max_width=150, icon='bomb', color='blue', group=None):
        gr = self.map
        if group:
            gr = folium.map.FeatureGroup(group)
            gr.add_to(self.map)
        if popup:
            for loc, pop in zip(location, popup):
                folium.Marker(location=loc, popup=folium.Popup(self.popup(pop), max_width=max_width), 
                              icon=folium.Icon(color=color, icon=icon, prefix='fa')).add_to(gr)
        else:
            for loc in location:
                folium.Marker(location=loc, icon=folium.Icon(color=color, icon=icon, prefix='fa')).add_to(gr)            
        
    def show(self, mapname=None):
        folium.LayerControl().add_to(self.map)
        if mapname:
            self.map.save(mapname)
        return self.map
    
    @staticmethod
    def popup(dic):
        pop = ''
        for key, val in dic.items():
            pop += '<div>' + '<b>' + str(key) + '</b>' + ' : ' + str(val) + '</div>'
        return pop    
        

In [114]:
a = {'test' : 3, 'truc':'machin'}
print(carto.popup(a))

<div>test : 3</div><div>truc : machin</div>


In [ ]:
'''mapf = folium.Map(location=[48.87, 2.35], zoom_start=6)
gr = folium.map.FeatureGroup('irve')
gr.add_to(mapf)
folium.Marker(location=[45.76, 4.83], popup=folium.Popup('texte', max_width=150), 
                              icon=folium.Icon(color='red')).add_to(gr)
folium.LayerControl().add_to(mapf)
folium.Marker(location=[43.3, 5.38], popup=folium.Popup('texte', max_width=150), 
                              icon=folium.Icon(color='green')).add_to(gr)
mapf'''

In [135]:
from json_ntv import NtvSingle, NtvList, NtvSet, Ntv, NtvConnector
from json_ntv import NtvType, Namespace
from datetime import datetime, date, time

location = [{'paris': [2.35, 48.87]}, [4.83, 45.76], [5.38, 43.3]]

cart = carto([48.87, 2.35])
cart.add_marker_list([[48.87, 2.35], [45.76, 4.83]], [{'paris' : 75}, {'lyon': 69}], color='green')#, group='bornes')
cart.add_marker_list([[43.3, 5.38]], color='red', group='IRVE', icon='bug')
#folium.LayerControl().add_to(cart.map)
cart.show()


In [70]:
tab_data   = {'value':           [10, 20, 30, 40],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21', '2022-01-22'],
              'value32::int32':  [10, 20, 30, 40],
              'coord::point':    [[1,2], [3,4], [5,6], [7,8]],
              'names::string':   ['john', 'eric', 'judith', 'max'],
              'index':           [1, 2, 3, 4],
              'complete_test':   [['a', 'b'], [0, 0, 1, 0]],
              'complete_date':   [{"::date": ["2000-01-01", "2000-02-01"]}, [0, 0, 1, 0]],
              'implicit_test':   [['a', 'b'], [1]],
              'implicit_test2':  [['a', 'b'], [2]],
              'unic_test':       'valunic' }
il = from_ntv_ilist(tab_data)
'''for idx in il.lindex:
    print('idx : ', idx.name, idx.codec, idx.keys)
for info in il.indexinfos():
    print('indexinfos : ', info)
print('\nil : ', il.lname, il.idxname, il.lvarname, il.primary) #, il.primaryname)
ntv = Ntv.obj(tab_data)
#print(ntv)
lidx = [list(decode_ntv_field(ntvf)) for ntvf in ntv]
print('\ntest', lidx, '\n\n')'''
for mode in ['full', 'default', 'optimize']:
    #print(to_ntv_ilist(il, mode))
    il2 = from_ntv_ilist(to_ntv_ilist(il, mode))
    print(mode, il == il2)
    #il2.analysis.actualize()
    if il != il2:
        print(to_ntv_ilist(il, mode))
        print(il.lvarname)
        print(il2.lvarname)

full True
default True
optimize True
